#RAG_LLM_PDF

In [1]:
import json
import yaml
import os

# Function to load config (expects JSON or YAML only)
# Function to load config (tries JSON first, then YAML)
def load_config(config_path="config.json"):
    for path in ["config.json", "config.yaml"]:  # Prioritizes JSON, then YAML
        if os.path.exists(path):  # Check if the file exists
            config_path = path  # Set config_path to the found file
            break
    else:  
        print("Config file not found. Using defaults.")
        return {"vector_db": "chromadb"}  # Default settings

    with open(config_path, "r") as file:
        return json.load(file) if config_path.endswith(".json") else yaml.safe_load(file)

# Load config
config = load_config()

# Required keys that may need API keys
KEY_REQUIRED = {"openai", "pinecone", "weaviate"}

# Assign config values with defaults
vector_db = config.get("vector_db", "chromadb")
vector_db_key = config.get("vector_db_key", None)
embedding_model = config.get("embedding_model", "sentence-transformers/all-MiniLM-L6-v2")
embedding_model_key = config.get("embedding_model_key", None)
llm_model = config.get("llm_model", "gpt-3.5-turbo")
llm_model_key = config.get("llm_model_key", None)

data_source = config.get("data_source", "pdf")
pdf_folder = config.get("pdf_folder", "data/pdfs")
text_folder = config.get("text_folder", "data/texts")
url_folder = config.get("url_folder", "data/urls")
history_output_folder = config.get("history_output_folder", "output/history")

# Check if vector_db requires a key but it's missing
if vector_db in KEY_REQUIRED and not vector_db_key:
    print(f"⚠️ Warning: {vector_db} requires an API key, but none provided!")

if llm_model in KEY_REQUIRED and not llm_model_key:
    print(f"⚠️ Warning: {llm_model} requires an API key, but none provided!")

# Print loaded settings
print("\n--- Loaded Configuration ---")
print(f"Vector Database: {vector_db} {'(Key Required)' if vector_db in KEY_REQUIRED else ''}")
print(f"Embedding Model: {embedding_model} {'(Key Required)' if embedding_model in KEY_REQUIRED else ''}")
print(f"LLM Model: {llm_model} {'(Key Required)' if llm_model in KEY_REQUIRED else ''}")
print(f"Data Source: {data_source}")
print(f"PDF Folder: {pdf_folder}")
print(f"Text Folder: {text_folder}")
print(f"URL Folder: {url_folder}")
print(f"History Output Folder: {history_output_folder}")

timestamp = __import__("datetime").datetime.now().strftime("%Y%m%d_%H%M%S")

print("\n ", timestamp)

# STEP 1 IS CLEAR - IT GATHERS THE CONFIG DETAILS 
#Load config.json (fallback to config.yaml).




--- Loaded Configuration ---
Vector Database: chromadb 
Embedding Model: sentence-transformers/all-MiniLM-L6-v2 
LLM Model: gpt-3.5-turbo 
Data Source: pdf
PDF Folder: data/pdfs
Text Folder: data/texts
URL Folder: data/urls
History Output Folder: output/history

  20250403_130136


In [2]:
# CHROMADB - LOOK AT THE VECTOR DB AND CLEAR IT IF YOU WISH TO

In [3]:
import chromadb

def clearFlushChromaDB():

    # Initialize the Chroma client
    chroma_client = chromadb.PersistentClient(path="chromadb_store")
    
    # Step 1: List existing collections
    collections = chroma_client.list_collections()
    print("Existing Collections:", collections)
    
    # Step 2: Specify which collections to delete (e.g., [1], [1, 2], etc.)
    collections_to_delete = [0,1,2]  # Adjust this based on the index you want to delete
    
    # Step 3: Delete the selected collections
    for idx in collections_to_delete:
        collections = chroma_client.list_collections()  # Re-fetch the list of collections
        if idx < len(collections):
            collection_name = collections[idx]
            print(f"Attempting to delete collection: {collection_name}")
            chroma_client.delete_collection(collection_name)
            print(f"Deleted collection: {collection_name}")
        else:
            print(f"Collection index {idx} is out of range. No deletion performed.")
    
    # Confirm all collections have been deleted
    remaining_collections = chroma_client.list_collections()
    print("Remaining Collections:", remaining_collections)

    return


#clearFlushChromaDB()  # uncomment this in order to clear the chromaDB , vector embeddings and rebuild it again
    


Existing Collections: ['mcs2025']
Attempting to delete collection: mcs2025
Deleted collection: mcs2025
Collection index 1 is out of range. No deletion performed.
Collection index 2 is out of range. No deletion performed.
Remaining Collections: []


In [7]:
# NOW LETS USE THE GENAI WAY OF LOADING THE DOCUMENT

In [4]:
import os
from langchain.document_loaders import PyPDFLoader

def load_pdf_documents(pdf_folder="data/pdfs"):
    """Loads all PDF files from the specified folder using PyPDFLoader and extracts text."""
    pdf_documents = {}

    # Iterate through each file in the folder
    for filename in os.listdir(pdf_folder):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            loader = PyPDFLoader(pdf_path)  
            documents = loader.load()  # Returns a list of Document objects

            # Extract text from each document and concatenate into a single string
            extracted_text = "\n".join([doc.page_content for doc in documents])

            pdf_documents[filename] = extracted_text  # Store text, not Document objects
            print(f"Loaded {len(documents)} pages from {filename}")
    
    print(f"✅ Total PDFs processed: {len(pdf_documents)}")
    return pdf_documents  # Dictionary {filename: "full text content"}

# Step 1: Extract text
extracted_texts = load_pdf_documents("data/pdfs")




Loaded 3 pages from CMO-Pink-Sheet-March-2025.pdf
Loaded 216 pages from mcs2025.pdf
✅ Total PDFs processed: 2


In [5]:
# CHUNK THE DATA


In [6]:

# CHUNK IT 

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(extracted_texts, chunk_size=1000, chunk_overlap=200):
    """Splits extracted text into chunks for better embedding."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunked_data = {}
    
    for filename, text in extracted_texts.items():
        chunks = text_splitter.split_text(text)
        chunked_data[filename] = chunks
    
    print(f"✅ Chunking completed. Processed {len(extracted_texts)} files.")
    return chunked_data  # Dictionary {filename: [chunks]}


# Step 1: Extract text (Assuming extracted_texts is already created)
chunked_data = chunk_text(extracted_texts)
print(f"Total PDFs processed: {len(chunked_data)}")

# STEP 3 THE DATA IS CHUNKED USING RecursiveCharacterTextSplitter, THIS IS CLEAR AND DONE
# - Splits the extracted text into smaller chunks for better processing.
# - Uses `RecursiveCharacterTextSplitter` from LangChain for efficient chunking.
# - Allows setting chunk size (default: 1000 characters) and overlap (default: 200 characters).
# - Processes each extracted PDF text and stores chunks in a dictionary.



✅ Chunking completed. Processed 2 files.
Total PDFs processed: 2


In [7]:
# CHROMADB EMBEDDING 

import chromadb
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from langchain_huggingface import HuggingFaceEmbeddings  # ✅ Correct import



def store_embeddings_in_chroma(chunked_data, persist_directory="chromadb_store"):
    """Embeds chunked text and stores it in ChromaDB."""
    # Initialize embedding model
    #embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    # Initialize ChromaDB client
    chroma_client = chromadb.PersistentClient(path=persist_directory)

    for filename, chunks in chunked_data.items():
        collection = chroma_client.get_or_create_collection(name=filename.replace(".pdf", ""))
        embeddings = embedding_model.embed_documents(chunks)

        for i, chunk in enumerate(chunks):
            collection.add(
                ids=[f"{filename}_{i}"],
                metadatas=[{"source": filename, "chunk_index": i}],
                documents=[chunk],
                embeddings=[embeddings[i]]
            )

    print(f"✅ Embeddings stored in ChromaDB at '{persist_directory}'.")
    return chroma_client


print("Functions for embedding >>")
# - `chromadb` → Manages the local vector database for efficient similarity search.
# - `langchain.vectorstores.Chroma` → Provides a LangChain wrapper for ChromaDB.
# - `langchain_huggingface.HuggingFaceEmbeddings` → Embeds text using a transformer model.
# - `PersistentClient` from `chromadb` → Ensures embeddings are stored persistently.


Functions for embedding >>


In [8]:
import time
start_time = time.time()  # Record start time


chroma_client = store_embeddings_in_chroma(chunked_data)
collections = chroma_client.list_collections()
print(f"🗂️ Total collections in ChromaDB: {len(collections)}")



end_time = time.time()  # Record end time
elapsed = end_time - start_time
print(f"Time taken to store embeddings in ChromaDB: {elapsed:.2f} seconds")



# STEP 4 THE DATA IS EMBEDDED  CROMA HUGGING FACE  , THIS IS CLEAR AND DONE..
 
# - Initializes the `HuggingFaceEmbeddings` model to convert text chunks into vector embeddings.
# - Uses `chromadb.PersistentClient` to create or retrieve a persistent ChromaDB collection.
# - Iterates over chunked text data and:
#   - Embeds each chunk using the transformer model.
#   - Stores embeddings in ChromaDB with metadata (source file, chunk index).
# - Logs confirmation once embeddings are successfully stored.
 

✅ Embeddings stored in ChromaDB at 'chromadb_store'.
🗂️ Total collections in ChromaDB: 2
Time taken to store embeddings in ChromaDB: 551.93 seconds


#CHECK TIME IT TOOK TO EMBED

In [ ]:
# DO CHROMA DB  SEEK 

In [10]:
import chromadb
from langchain_huggingface import HuggingFaceEmbeddings

def queryChromaDB(user_query, persist_directory="chromadb_store", top_k=3):
    """Searches ChromaDB for relevant text chunks based on user query."""
    # Load embedding model
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    query_embedding = embedding_model.embed_query(user_query)
    
    # Initialize ChromaDB client
    chroma_client = chromadb.PersistentClient(path=persist_directory)
    collections = chroma_client.list_collections()
    
    if not collections:
        print("⚠️ No collections found in ChromaDB. Ensure embeddings are stored first.")
        return None
    
    # Search across all collections
    results = []
    for collection_name in collections:
        collection = chroma_client.get_collection(collection_name)
        #search_results = collection.query(queries=[query_embedding], n_results=top_k)
        search_results = collection.query(query_embeddings=[query_embedding], n_results=top_k)

        
        for doc, metadata, distance in zip(search_results['documents'][0], search_results['metadatas'][0], search_results['distances'][0]):
            results.append((doc, metadata, distance))
    
    # Sort results by similarity (lower distance is better)
    results.sort(key=lambda x: x[2])  # Sort by distance (ascending)
    
    # Display top results
    print("\n🔍 Query Results:")
    for i, (text, metadata, distance) in enumerate(results[:top_k]):
        print(f"\nResult {i+1}:\nSource: {metadata['source']} (Chunk {metadata['chunk_index']})\nDistance: {distance:.4f}\n{text}\n")
    
    return results[:top_k]  # Return top matches


user_query = "Tell me something unique about commodities"
query_results = queryChromaDB(user_query, persist_directory="chromadb_store", top_k=3)
print(query_results)



🔍 Query Results:

Result 1:
Source: mcs2025.pdf (Chunk 5)
Distance: 0.9069
U.S. Geological Survey, 2025, Mineral commodity summaries 2025 (ver. 1.2, March 2025): U.S. Geological Survey, 212 p., 
https://doi.org/10.3133/mcs2025. 
Associated data for this publication: 
U.S. Geological Survey, 2025, Data release for mineral commodity summaries 2025: U.S. Geological Survey data release, 
https://doi.org/10.5066/P13XCP3R. 
ISBN  978-1-4113-4595-9
CONTENTS 
General: Page Page 
Introduction .................................................................... 3 
Figure 1—The Role of Nonfuel Mineral Commodities in 
the U.S. Economy ....................................................... 4 
Significant Events, Trends, and Issues .......................... 5 
Figure 2—2024 U.S. Net Import Reliance ..................... 7 
Figure 3—Leading Import Sources (2020–23) of  
Nonfuel Mineral Commodities .................................... 8 
Table 1—U.S. Mineral Industry Trends ........................

In [12]:
import textwrap

def format_query_results_for_llm(query_results, user_query, max_chars=1000):
    """
    Trims and formats the ChromaDB query results before sending to an LLM.
    Ensures the content is concise, relevant, and within token limits.

    Parameters:
    - query_results: List of (text, metadata, distance) tuples from ChromaDB.
    - user_query: The original user query string.
    - max_chars: Maximum length of retrieved text to pass to the LLM.

    Returns:
    - formatted_prompt: The final structured prompt for the LLM.
    """
    formatted_text = []
    total_length = 0

    for i, (text, metadata, distance) in enumerate(query_results):
        source = metadata.get("source", "Unknown Source")
        chunk_index = metadata.get("chunk_index", "N/A")

        # Trim the content but maintain meaningful sentences
        trimmed_text = textwrap.shorten(text, width=300, placeholder="...")

        # Ensure sentences are not cut off abruptly (extend to nearest period)
        if len(trimmed_text) < len(text):
            last_period = trimmed_text.rfind(".")
            if last_period != -1:
                trimmed_text = trimmed_text[: last_period + 1]

        entry = f"🔹 **Source:** {source} (Chunk {chunk_index})\n   {trimmed_text}"
        total_length += len(entry)

        if total_length > max_chars:
            break  # Stop if we exceed the limit

        formatted_text.append(entry)

    # Construct the final structured prompt
    formatted_prompt = (
        "You are an expert in finance and commodities. Based on the following retrieved information, "
        "provide a unique insight about commodities.\n\n"
        f"📌 **User Query:** \"{user_query}\"\n\n"
        "🔎 **Retrieved Information:**\n"
        + "\n\n".join(formatted_text) +
        "\n\n📝 **Task:** Summarize key insights from the retrieved information and provide a unique perspective about commodities."
    )

    return formatted_prompt

# Example usage:
user_query = "Tell me something unique about commodities"
formatted_prompt = format_query_results_for_llm(query_results, user_query)
print(formatted_prompt)


You are an expert in finance and commodities. Based on the following retrieved information, provide a unique insight about commodities.

📌 **User Query:** "Tell me something unique about commodities"

🔎 **Retrieved Information:**
🔹 **Source:** mcs2025.pdf (Chunk 5)
   U.S. Geological Survey, 2025, Mineral commodity summaries 2025 (ver. 1.2, March 2025): U.S. Geological Survey, 212 p., https://doi.org/10.3133/mcs2025. Associated data for this publication: U.S. Geological Survey, 2025, Data release for mineral commodity summaries 2025: U.S. Geological Survey...

🔹 **Source:** CMO-Pink-Sheet-March-2025.pdf (Chunk 8)
   Sugar, U.S. $/kg b/ 0.79 0.89 0.84 0.95 0.89 0.84 0.81 0.83 0.81 0.80 0.82 Sugar, World $/kg b/ 0.41 0.52 0.45 0.54 0.49 0.43 0.43 0.45 0.44 0.40 0.42 continued on next page Annual Averages Quarterly Averages March 4, 2025 Monthly Averages - 1 - http://www.worldbank.org/commodities World Bank...

📝 **Task:** Summarize key insights from the retrieved information and provide 

In [ ]:
import pickle

# Assuming 'formatted_prompt' variable holds the output of format_query_results_for_llm

# Save the formatted prompt to a pickle file
output_file = "formatted_prompt.pkl"
with open(output_file, 'wb') as f:
    pickle.dump(formatted_prompt, f)

print(f"Formatted prompt saved to: {output_file}")

# --- Later in your code or in a different Colab session ---

# Retrieve the formatted prompt from the pickle file
loaded_prompt = None
input_file = "formatted_prompt.pkl"
try:
    with open(input_file, 'rb') as f:
        loaded_prompt = pickle.load(f)
    print(f"Formatted prompt loaded from: {input_file}")
except FileNotFoundError:
    print(f"Error: File not found at {input_file}")

# Now you can use 'loaded_prompt' as your formatted prompt
# For example:
# if loaded_prompt:
#     response = llm_pipeline(loaded_prompt, max_length=512, temperature=0.7, do_sample=True)
#     print("🤖 LLM Response:\n", response[0]["generated_text"])

In [ ]:
# This model worked on COLAB

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Load a smaller, open-source model suitable for Colab
MODEL_NAME = "google/flan-t5-base"

# Load the tokenizer & model (force CPU usage - adjust if using GPU)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, device_map="cpu", torch_dtype="auto")

# Create text generation pipeline (for T5 models, it's often 'text2text-generation')
llm_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Assuming you have run queryChromaDB and have query_results
# Assuming you have your user_query defined
formatted_prompt = format_query_results_for_llm(query_results, user_query)

# Generate response from LLM
response = llm_pipeline(formatted_prompt, max_length=512, temperature=0.7, do_sample=True)

# Print final output
print("🤖 LLM Response:\n", response[0]["generated_text"])

# the LLM Response on COLAB 
# The World Bank's  Global Value Indexes for the month of March is a reliable measure of the value of U.S. commodity and 
#of the commodity markets from the year of its creation in the 1990s to the present.


In [13]:
# Alternate models , are not working on my laptop CPU , but will work on colab or in systems with GPU 

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the model (Change to "HuggingFaceH4/zephyr-7b-alpha" if needed)
MODEL_NAME = "mistralai/Mistral-7B-Instruct"

# Load the tokenizer & model (force CPU usage)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="cpu", torch_dtype="auto")

# Create text generation pipeline

llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Your formatted prompt
formatted_prompt = format_query_results_for_llm(query_results, user_query)

# Generate response from LLM
response = llm_pipeline(formatted_prompt, max_length=1024, temperature=0.7, do_sample=True)

# Print final output
print("🤖 LLM Response:\n", response[0]["generated_text"])


OSError: mistralai/Mistral-7B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
#Zephyr-7B (HuggingFaceH4) is a large model
# Use a smaller model: Try mistral-7b-instruct or mistral-7b,
# cloud api API like OpenAI (GPT-4-turbo) or Hugging Face’s inference endpoints
# GGUF quantized models: With llama.cpp
# 

#OPEN AI DOES NOT WORK ONLY BECAUSE OF THE USAGE LIMITATIONS I HAVE... 
#SO CHECK NEXT CELL - HUGGING FACE FREE OPENSOURCE
import openai
import os
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb

#Load OpenAI API Key

OPENAI_API_KEY = "get the key"  




import openai

#Load OpenAI API Key
 
def query_openai_llm(query):
    """Fetches relevant context and queries OpenAI's GPT-3.5-turbo."""
    context = retrieve_context(query)  # Ensure you have this function defined
    prompt = f"""
    You are an AI assistant for answering finance-related queries. Below is the context:

    {context}

    Please provide an accurate and concise answer based on the context above.
    """

    client = openai.OpenAI(api_key=OPENAI_API_KEY)  # Create OpenAI client

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Free-tier supports only GPT-3.5-Turbo
        messages=[
            {"role": "system", "content": "You are a financial assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# Example query
test_query = "What is commodity generally speaking?"
response = query_openai_llm(test_query)
print("Generated Answer:", response)


In [20]:
# NOW TRYING TO DO LLM USING  

In [ ]:
#is awaiting a review from the repo authors ........Have made a request at HG for access...
this is loading and then hanging the server..........

from transformers import pipeline
from huggingface_hub import login

# Login with your Hugging Face API key

HF_TOKEN ="get a key......."

login(HF_TOKEN)

# Use Facebook's LLaMA 2 model from Hugging Face (No download needed!)
qa_pipeline = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf")

# Ask a question
query = "What are the latest advancements in AI?"
response = qa_pipeline(query, max_length=256)

print("\n💡 AI Response:\n", response[0]['generated_text'])


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

D:\Anaconda_Python\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from huggingface_hub import login
from langchain_chroma import Chroma  
from langchain_huggingface import HuggingFaceEmbeddings

# Step 1: Authenticate with Hugging Face
HF_TOKEN = "get a key.."
login(HF_TOKEN)

# Step 2: Initialize LLaMA 2 model
qa_pipeline = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf")

# Step 3: Load ChromaDB with stored embeddings
persist_directory = "chromadb_store"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

def retrieve_relevant_chunks(query, k=3):
    """Retrieve the top-k most relevant chunks from ChromaDB"""
    docs = vector_db.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

def query_rag_system(user_query):
    """Queries ChromaDB, constructs an enhanced prompt, and gets LLM response"""
    
    # Retrieve relevant text from stored embeddings
    retrieved_chunks = retrieve_relevant_chunks(user_query)
    context = "\n".join(retrieved_chunks)  

    # Step 4: Print enhanced prompt
    enhanced_prompt = f"Context:\n{context}\n\nUser Query:\n{user_query}"
    print("\n🔹 Enhanced Prompt:\n", enhanced_prompt)

    # Step 5: Query LLaMA 2 model
    response = qa_pipeline(enhanced_prompt, max_length=512, do_sample=True)
    
    return response[0]['generated_text']

# Run the RAG system
user_query = input("Enter your query: ")
response = query_rag_system(user_query)

print("\n💡 AI Response:\n", response)
